<a href="https://colab.research.google.com/github/Wishmiii/Customer-Churn-Prediction-System-ML/blob/main/CustomerChurnPredictionSystem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [30]:
!ls "/content/drive/MyDrive/Colab_Notebooks/MLCW"

CustomerChurnPredictionSystem.ipynb  WA_Fn-UseC_-Telco-Customer-Churn.csv


In [31]:
import os, sys
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import (accuracy_score,precision_score,recall_score,f1_score,
                             roc_auc_score, classification_report, confusion_matrix,
                             mean_squared_error, r2_score)
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
import joblib
import tensorflow as tf
from imblearn.over_sampling import SMOTE

In [102]:
df=pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/MLCW/WA_Fn-UseC_-Telco-Customer-Churn.csv')

# Task 01 - Exploratory Data Analysis and Visualization

##Dataset Dimensions & Metadata

In [ ]:
df.head()

In [ ]:
print("Rows, Columns =",df.shape)


In [ ]:
# Was not tought in tut
df.columns.tolist()

In [ ]:
# Basic Checks
df.info()


In [ ]:
df.describe(include="all")

##Identifying Missing Values

In [ ]:
# Identifying missing values
df.isnull().sum().sort_values(ascending=False)

In [ ]:
# Check for whitespace-only cells
(df==" ").sum().sort_values(ascending=False)

In [ ]:
# Detect cells that can not be converted to numbers in the TotalCharges column where missing values were
print(pd.to_numeric(df['TotalCharges'],errors='coerce').isnull().sum())

In [ ]:
# Identifying categorical and numerical values
print(f"Categorical Value Columns : {df.select_dtypes(include='object').columns.tolist()}")
print(f"Numeric Value Columns : {df.select_dtypes(include=['float64','int64']).columns.tolist()}")



In [ ]:
# Was not tought in tut
df.duplicated().sum()

In [ ]:
# Was not tought in tut
df.dtypes

In [99]:
(df['InternetService'].unique())

array(['DSL', 'Fiber optic', 'No'], dtype=object)

In [100]:
(df['Contract'].unique())

array(['Month-to-month', 'One year', 'Two year'], dtype=object)

In [101]:
df['PaymentMethod'].unique()

array(['Electronic check', 'Mailed check', 'Bank transfer (automatic)',
       'Credit card (automatic)'], dtype=object)

In [ ]:
df['MultipleLines'].unique()

In [ ]:
df['OnlineBackup'].unique()

In [ ]:
df['OnlineSecurity'].unique()

In [ ]:
df['DeviceProtection'].unique()

In [ ]:
df['TechSupport'].unique()

In [ ]:
df['StreamingTV'].unique()

In [ ]:
(df['StreamingMovies'].unique())

In [ ]:
(df['Contract'].unique())

In [ ]:
(df['PaperlessBilling'].unique())

In [ ]:
(df['PaymentMethod'].unique())

In [ ]:
(df['SeniorCitizen'].unique())

In [ ]:
(df['Churn'].unique())

In [ ]:
(df['StreamingMovies'].unique())

##Feature Distributions

###Numerical Feature Distributions

In [ ]:
#Correlation Matrix
plt.figure(figsize=(10,6))
numeric_features = df.select_dtypes(include=['int64', 'float64'])
corr = numeric_features.corr()
sns.heatmap(corr, annot=True, cmap='coolwarm')
plt.title('Correlation Matrix of Numerical Variables')
plt.show()

In [ ]:
plt.figure(figsize=(6,4))
sns.histplot(df['tenure'], bins=30, color='skyblue', kde=True)
# sns.histplot(x='tenure', data=df, hue='Churn', bins=30, palette='pastel', kde=True)
plt.title('Tenure Distribution')
plt.xlabel('Tenure (Months)')
plt.ylabel('Customer Count')
plt.show()

In [ ]:
plt.figure(figsize=(6,4))
sns.histplot(df['MonthlyCharges'], bins=30, color='skyblue', kde=True)
# sns.histplot(x='tenure', data=df, hue='Churn', bins=30, palette='pastel', kde=True)
plt.title('Distribution of Monthly Charges ')
plt.xlabel('Monthly Charges ($)')
plt.ylabel('Customer Count')
plt.show()

In [ ]:
#Used bar chart becuase 'TotalCharges' is in 'object' data type
total_charges=pd.to_numeric(df['TotalCharges'], errors='coerce')
plt.figure(figsize=(8,5))
bins=[0,500, 1000,1500, 2000, 2500, 3000, 3500, 4000, 4500, 5000, 5500,6000, 6500, 7000, 7500,8000, 8500, 9000, 9500,10000]
sns.histplot(total_charges,bins= bins, color='skyblue', kde=True)
plt.xticks(rotation=45)
plt.title("Distribution of TotalCharges ")
plt.xlabel("Charge Ranges")
plt.ylabel("Customer Count")
plt.show()

In [ ]:
#Boxplots of Numeric Variables
numerical_cols=['tenure', 'MonthlyCharges', 'TotalCharges']
plt.figure(figsize=(20,20))
for i, col in enumerate(numerical_cols):
  plt.subplot(4,4,i+1)
  series = pd.to_numeric( df[col], errors='coerce')
  sns.boxplot(y=series, color=sns.color_palette("pastel",8)[0])
  plt.title(f'{col} Distribution')
  plt.ylabel(col)

plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(6,4))
sns.boxplot(y=df['MonthlyCharges'], color='skyblue')
plt.title('Distribution of Monthly Charges')
plt.ylabel('Monthly Charges $')
plt.show()

In [ ]:
plt.figure(figsize=(6,4))
sns.boxplot(y=pd.to_numeric(df['TotalCharges'], errors='coerce'), color='skyblue')
plt.title('Distribution of Total Charges')
plt.ylabel('Total Charges $')
plt.show()



In [ ]:
plt.figure(figsize=(6,4))
sns.boxplot(y=df['tenure'], color='skyblue')
plt.title('Distribution of Tenure')
plt.ylabel('Tenure (Months)')
plt.show()

In [ ]:
categorical_cols =  [ 'gender', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod', 'Churn']

In [ ]:
plt.figure(figsize=(20,20))
for i, col in enumerate(categorical_cols):
  plt.subplot(5,4,i+1)
  sns.countplot(x=col, data=df, color=sns.color_palette("pastel",8)[0])
  plt.title(f'{col} Distribution')
  plt.xlabel(col)
  plt.ylabel('Customer Count')

plt.tight_layout()
plt.show()


##Categorical Data Distribution

In [ ]:
sns.color_palette("pastel",8)

In [ ]:
plt.figure(figsize=(6,4))
sns.countplot(x='Contract',data=df, color=sns.color_palette("pastel",8)[0])
plt.title('Contract Type Distribution')
plt.xlabel('Contract')
plt.ylabel('Customer Count')
plt.show()

In [ ]:
plt.figure(figsize=(6,4))
sns.countplot(x='Contract',data=df, color=sns.color_palette("pastel",8)[0])
plt.title('Contract Type Distribution')
plt.xlabel('Contract')
plt.ylabel('Customer Count')
plt.show()

In [ ]:
plt.figure(figsize=(6,4))
sns.countplot(x='Contract',data=df, color=sns.color_palette("pastel",8)[0])
plt.title('Contract Type Distribution')
plt.xlabel('Contract')
plt.ylabel('Customer Count')
plt.show()

###Key Feature Relationships

In [ ]:
# Contract Vs. Churn
plt.figure(figsize=(6,4))
sns.countplot(x='Contract',data=df, hue='Churn', palette='pastel')
plt.title('Churn by Contract Type')
plt.xlabel('Contract')
plt.ylabel('Customer Count')
plt.show()

In [ ]:
#Internet Service Vs. Churn
plt.figure(figsize=(6,4))
sns.countplot(x='InternetService',data=df, hue='Churn', palette='pastel')
plt.title('Churn by Internet Service')
plt.xlabel('Internet Service')
plt.ylabel('Customer Count')

In [ ]:
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')

In [ ]:
#Total Charges Vs. Churn
plt.figure(figsize=(6,4))
sns.boxplot(x='Churn',y='TotalCharges', data=df, hue='Churn', palette='pastel')
plt.title('Churn by Total Charges')
plt.xlabel('Total Charges')
plt.ylabel('Customer Count')

##Data Visualization

In [ ]:
# Visualizing the Distribution of Churn
# Checks the class balance
plt.figure(figsize=(6,4))
sns.countplot(x='Churn',data=df, hue='Churn', palette='pastel')
plt.title('Churn Distribution')
plt.xlabel('Churn')
plt.ylabel('Customer Count')
plt.show()

In [ ]:
# Calculate the precentage of churn values
df['Churn'].value_counts(normalize=True)*100

In [ ]:
plt.figure(figsize=(6,4))
sns.countplot(data=df,x="Churn",palette="Set2")
plt.title("Churn Distribution")
plt.show()

df['Churn'].value_counts(normalize=True)*100

In [ ]:
#Total Charges Vs. Churn
plt.figure(figsize=(6,4))
sns.boxplot(x='Churn',y='tenure', data=df, hue='Churn', palette='pastel')
plt.title('Churn by Tenure')
plt.xlabel('Tenure')
plt.ylabel('Customer Count')
plt.show()

In [ ]:
# Contract Type vs Churn
plt.figure(figsize=(6,4))
sns.countplot(x='Contract',data=df, hue='Churn', palette='pastel')
plt.title('Churn by Contract Type')
plt.xlabel('Contract')
plt.ylabel('Customer Count')
plt.show()

In [ ]:
# Seniority vs Churn
plt.figure(figsize=(6,4))
sns.countplot(x='SeniorCitizen',data=df, hue='Churn', palette='pastel')
plt.title('Churn by Seniority')
plt.xlabel('Seniority')
plt.ylabel('Customer Count')
plt.show()

In [ ]:
# Churn Rate by Seniority (HeatMap)
senior_ct = pd.crosstab(df['SeniorCitizen'], df['Churn'], normalize="index")*100
plt.figure(figsize=(6,4))
sns.heatmap(senior_ct, annot=True, cmap="coolwarm", fmt=".1f")
plt.title("Churn Rate by Seniority (%)")
plt.show()

In [ ]:
plt.figure(figsize=(8,5))
sns.violinplot(data=df, x="Churn", y="tenure", palette="pastel")
plt.title("Tenure Distribution by Churn Status (Violin Plot)")
plt.xlabel("Churn")
plt.ylabel("Tenure (Months)")
plt.show()


In [ ]:
plt.figure(figsize=(8,5))
sns.kdeplot(df[df['Churn']=="Yes"]["tenure"], shade=True, color="red", label="Churned")
sns.kdeplot(df[df['Churn']=="No"]["tenure"], shade=True, color="green", label="Stayed")
plt.title("KDE Density Plot – Tenure by Churn Status")
plt.xlabel("Tenure (Months)")
plt.ylabel("Density")
plt.legend()
plt.show()


In [ ]:
plt.figure(figsize=(10,5))
sns.histplot(df[df["Churn"]=="Yes"]["tenure"], kde=True, bins=30, color="red", label="Churned")
sns.histplot(df[df["Churn"]=="No"]["tenure"], kde=True, bins=30, color="green", label="Stayed")
plt.title("Histogram Overlay – Tenure Distribution for Churn vs Non-Churn")
plt.xlabel("Tenure (Months)")
plt.ylabel("Frequency")
plt.legend()
plt.show()


In [ ]:
df['MultipleLines'].value_counts(normalize=True)*100

In [ ]:
#According to the above output 'No phone service" accounts for 9.7% of the data
# which is a significant portion (more than 5%). So we can use the method of creating
# a new category for this missing value

In [ ]:
#Check how churn behaves for each category
pd.crosstab(df['MultipleLines'],df['Churn'])

In [ ]:
# pd.crosstab(df['OnlineSecurity'],df['Churn'])
df['OnlineSecurity'].value_counts(normalize=True)*100

In [ ]:
# pd.crosstab(df['OnlineBackup'],df['Churn'])
df['OnlineBackup'].value_counts(normalize=True)*100

In [ ]:
# pd.crosstab(df['DeviceProtection'],df['Churn'])
df['DeviceProtection'].value_counts(normalize=True)*100

In [ ]:
# pd.crosstab(df['TechSupport'],df['Churn'])
df['TechSupport'].value_counts(normalize=True)*100

In [ ]:
# pd.crosstab(df['StreamingTV'],df['Churn'])
df['StreamingTV'].value_counts(normalize=True)*100

In [ ]:
# pd.crosstab(df['StreamingMovies'],df['Churn'])
df['StreamingMovies'].value_counts(normalize=True)*100

In [ ]:
churn_by_mult = pd.crosstab(df['MultipleLines'], df['Churn'], normalize='index') * 100
churn_by_mult


In [ ]:
plt.figure(figsize=(6,4))
sns.barplot(x=churn_by_mult.index, y=churn_by_mult['Yes'])
plt.ylabel("Churn Rate (%)")
plt.title("Churn Rate by MultipleLines Category")
plt.show()

# Task 02




Q: Implement the classification task using a Neural network-based model and a Decision Tree model. Conduct
Data preprocessing tasks required for the dataset, problem, and the algorithms. Perform hyperparameter
tuning required for the algorithms. Include all the information regarding your work and results in the report.
Respective libraries, frameworks, tools, etc., must be used for model implementation. The implemented models
should be compared based on the optimal evaluation metrics. Experimental results should be showcased for
both model experimental settings.

List
 - Data Preprocessing for the dataset, problem and algorithm
 - classification using NN and Decision Tree
 - Perform hyperparameter tuning for the algorithms
 - Compare models


##Handling Missing Values and Hidden Missing values

In [103]:
# Handling missing values in TotalCharges

# 1. Converting TotalCharges to a numerical value
df['TotalCharges'] = pd.to_numeric(df.TotalCharges,errors='coerce')
# This converts blanks to NaN

# According to above results there are 11 null values in TotalCharges column
# Those 11 values where imputed with the median of the column

df['TotalCharges'].fillna(df['TotalCharges'].median(), inplace=True)

/tmp/ipython-input-2848153744.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['TotalCharges'].fillna(df['TotalCharges'].median(), inplace=True)


In [104]:
#Search the entire dataset for blank strings, empty strings, cells with only whitespace
#and convert in to NaN values that are detectable in df.isnull() and df.isna()
df.replace(r'^\s*$',np.nan, regex=True, inplace=True)
df.isna().sum()
df.isnull().sum()

,0
customerID,0
gender,0
SeniorCitizen,0
Partner,0
Dependents,0
tenure,0
PhoneService,0
MultipleLines,0
InternetService,0
OnlineSecurity,0


##Remove Duplicate Coloumns

In [105]:
#Remove duplicate columns if there is any
df.drop_duplicates(inplace=True)

##Droping Unwanted Columns

In [106]:
# Removing 'customerID' column because it is just an identifier and not a feature
df.drop(columns=["customerID"], inplace=True)

In [107]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gender            7043 non-null   object 
 1   SeniorCitizen     7043 non-null   int64  
 2   Partner           7043 non-null   object 
 3   Dependents        7043 non-null   object 
 4   tenure            7043 non-null   int64  
 5   PhoneService      7043 non-null   object 
 6   MultipleLines     7043 non-null   object 
 7   InternetService   7043 non-null   object 
 8   OnlineSecurity    7043 non-null   object 
 9   OnlineBackup      7043 non-null   object 
 10  DeviceProtection  7043 non-null   object 
 11  TechSupport       7043 non-null   object 
 12  StreamingTV       7043 non-null   object 
 13  StreamingMovies   7043 non-null   object 
 14  Contract          7043 non-null   object 
 15  PaperlessBilling  7043 non-null   object 
 16  PaymentMethod     7043 non-null   object 


##Outlier Treatment

##Handling Traget Variables

In [108]:
#Encoding the Target variable
# Converting the predictor variable in to a binary values
# Maping 'Yes' and 'No' values to 1 and 0 and any other value to NaN and
# assigning it to the original column
df['Churn']= df['Churn'].map({'Yes':1, 'No':0})

##Data Transformation

###Encoding Categorical Features

In [109]:
#Converting Yes/No columns to binary
binary_columns = ['Partner', 'Dependents', 'PhoneService', 'MultipleLines', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'PaperlessBilling']

for column in binary_columns:
  df[column] = df[column].map({'Yes':1, 'No':0, 'No phone service':0, 'No internet service':0})

#mapping gender column
df['gender'] = df['gender'].map({'Female':1, 'Male':0})


In [110]:
#One-Hot-Encoding multi category columns
multi_cat_columns=['InternetService','Contract','PaymentMethod']
df=pd.get_dummies(df,columns=multi_cat_columns,drop_first=True)

In [111]:
X = df.drop(columns=['Churn'])
y=df['Churn']

In [112]:
df.isna().sum()

,0
gender,0
SeniorCitizen,0
Partner,0
Dependents,0
tenure,0
PhoneService,0
MultipleLines,0
OnlineSecurity,0
OnlineBackup,0
DeviceProtection,0


##Standardization


###Build Preprocessing Pipelines for both models

###Preprocessing Transformer

In [113]:
numeric_columns=['tenure','MonthlyCharges','TotalCharges']
categorical_columns=[col for col in X.columns if col not in numeric_columns]

preprocess = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_columns),
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_columns)
    ],
    remainder='drop'
)

##Train-Test Split

In [115]:
#Data should be splitted before balancing with SMOTE
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

##Handle Class Imbalance

In [117]:
X_train_preprocessed = preprocess.fit_transform(X_train)

In [118]:
#For Neural Network
#Apply SMOTE on training data
smote = SMOTE(random_state=42)
X_train_sm, y_train_sm = smote.fit_resample(X_train, y_train)